<a href="https://colab.research.google.com/github/Jalam1001/huggingface/blob/main/Copy_of_LFM2_MoE_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -qqq git+https://github.com/huggingface/transformers.git@0c9a72e4576fe4c84077f066e585129c97bfd4e6 --progress-bar off

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_id = "LiquidAI/LFM2-8B-A1B"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype="bfloat16",
#    attn_implementation="flash_attention_2" <- uncomment on compatible GPU
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Generate answer
prompt = "What is C. elegans?"
input_ids = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=True,
).to(model.device)

output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.3,
    min_p=0.15,
    repetition_penalty=1.05,
    max_new_tokens=512,
)

print(tokenizer.decode(output[0], skip_special_tokens=False))

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

<|startoftext|><|im_start|>user
What is C. elegans?<|im_end|>
<|im_start|>assistant
**C. elegans**, short for *Caenorhabditis elegans*, is a small, transparent nematode (roundworm) that has become one of the most important model organisms in biological research. Here’s a detailed breakdown:

### Key Characteristics:
1. **Size & Appearance**:  
   - Approximately 1 millimeter long, with a simple, elongated body and a fixed number of cells (959 somatic cells in adults).  
   - Transparent body allows direct observation of internal processes under a microscope.

2. **Life Cycle**:  
   - Short lifespan (~2–3 weeks), enabling rapid generational studies.  
   - Reproduces quickly via sexual or asexual reproduction (hermaphrodites produce offspring without mating).

3. **Genetic Simplicity**:  
   - Has a fully mapped genome (~100 million base pairs, ~20,000 genes).  
   - Well-characterized genetics due to conserved signaling pathways shared with humans.

4. **Behavior**:  
   - Displays co

In [8]:
def ask(prompt_text, temperature=0.4, max_new_tokens=512):
    """Generate a clear formatted response from the preloaded model."""
    input_ids = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt_text}],
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    output = model.generate(
        input_ids,
        do_sample=True,
        temperature=temperature,
        min_p=0.15,
        repetition_penalty=1.05,
        max_new_tokens=max_new_tokens,
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True).strip()
    print("🧠 Model Response:\n")
    print(response)

In [9]:
# Example usage:
prompt = "Explain the difference between photosynthesis and cellular respiration."
ask(prompt)

🧠 Model Response:

user
Explain the difference between photosynthesis and cellular respiration.
assistant
**Photosynthesis** and **cellular respiration** are two fundamental biological processes that are essentially opposite reactions in the energy flow of living organisms. Here's a clear breakdown of their differences:

---

### **1. Purpose / Function**
- **Photosynthesis**:  
  - Occurs in **plants, algae, and some bacteria**.  
  - Converts **light energy into chemical energy** stored in glucose.  
  - Produces **oxygen** as a byproduct.

- **Cellular Respiration**:  
  - Occurs in **all living cells** (plants, animals, fungi, protists).  
  - Breaks down **glucose to release energy** for cellular activities.  
  - Consumes **oxygen and releases carbon dioxide**.

---

### **2. Location in the Cell**
- **Photosynthesis**:  
  - Takes place in **chloroplasts**, specifically in the thylakoid membranes and stroma.

- **Cellular Respiration**:  
  - Occurs in the **mitochondria** (main

In [10]:
from IPython.display import display, Markdown

def ask(prompt_text, temperature=0.4, max_new_tokens=512):
    """Generate and display a clean, readable response from the preloaded model."""
    input_ids = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt_text}],
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    output = model.generate(
        input_ids,
        do_sample=True,
        temperature=temperature,
        min_p=0.15,
        repetition_penalty=1.05,
        max_new_tokens=max_new_tokens,
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Clean formatting: remove excessive Markdown symbols for better readability
    cleaned = (
        response.replace("**", "")
                .replace("###", "")
                .replace("---", "")
                .replace("|", " | ")
    )

    # Display with nice formatting
    display(Markdown(f"### 🧠 Model Response\n\n{cleaned}"))

In [11]:
prompt = "Explain the difference between photosynthesis and cellular respiration."
ask(prompt)

### 🧠 Model Response

user
Explain the difference between photosynthesis and cellular respiration.
assistant
Photosynthesis and cellular respiration are two fundamental biological processes that are essentially opposite in function, though they are interconnected in the global flow of energy on Earth.

Photosynthesis occurs primarily in plants, algae, and some bacteria. It is the process by which these organisms convert light energy from the sun into chemical energy stored in glucose. During photosynthesis, carbon dioxide (CO₂) and water (H₂O) are used, with sunlight providing the energy to produce glucose (C₆H₁₂O₆) and release oxygen (O₂) as a byproduct. The overall equation is:  
6CO₂ + 6H₂O + light energy → C₆H₁₂O₆ + 6O₂.

Cellular respiration, on the other hand, takes place in the cells of almost all living organisms, including plants, animals, fungi, and protists. It is the process by which cells break down glucose (and other organic molecules) in the presence of oxygen to produce usable energy in the form of ATP (adenosine triphosphate). Carbon dioxide and water are released as waste products. The simplified equation is:  
C₆H₁₂O₆ + 6O₂ → 6CO₂ + 6H₂O + ATP (energy).

In summary, photosynthesis stores energy by building glucose using sunlight, while cellular respiration releases energy by breaking down glucose using oxygen. Together, they form a cycle that sustains life by balancing oxygen and carbon dioxide in the atmosphere and providing energy for organisms.

## New Section
few test prompts


In [12]:
# 🔬 Single precision test for reasoning + knowledge
prompt = (
    "Explain how mitochondria convert food into usable energy for the cell. "
    "Start from glucose breakdown, describe the electron transport chain, "
    "and end with how ATP is produced. Keep it simple but detailed."
)
ask(prompt)

### 🧠 Model Response

user
Explain how mitochondria convert food into usable energy for the cell. Start from glucose breakdown, describe the electron transport chain, and end with how ATP is produced. Keep it simple but detailed.
assistant
Certainly! Here’s a clear step-by-step explanation of how mitochondria convert food—like glucose—into usable energy (ATP) in the cell:



1. Breaking Down Glucose (Cellular Respiration)  
The process starts with glucose, a sugar obtained from food, entering the mitochondria. Inside, glucose is broken down in several stages, primarily through glycolysis (in the cytoplasm), then the Krebs cycle (in the mitochondrial matrix).  

- Glycolysis splits one glucose molecule into two molecules of pyruvate, producing a small amount of ATP and NADH (an electron carrier).  
- The pyruvate enters the mitochondria and is converted into acetyl-CoA, which feeds into the Krebs cycle (also called the citric acid cycle).  
- During the Krebs cycle, pyruvate is fully broken down, releasing carbon dioxide and generating more electron carriers (NADH and FADH₂) while generating a tiny amount of ATP directly.



2. The Electron Transport Chain (ETC)  
The real power comes from the electron transport chain, located on the inner mitochondrial membrane. Here’s how it works:  

- High-energy electrons from NADH and FADH₂ are passed along protein complexes in the membrane.  
- As electrons move through these complexes, they release energy used to pump protons (H⁺ ions) from the mitochondrial matrix into the space between membranes, creating a proton gradient (a sort of stored energy).  
- This gradient drives protons back across the membrane through a key enzyme called ATP synthase.



3. Making ATP  
The flow of protons through ATP synthase powers the synthesis of ATP—the cell’s main energy currency.  

- As protons rush back into the matrix, their movement pushes a rotor in ATP synthase, triggering a chemical reaction that joins ADP and inorganic phosphate (Pi) to form ATP.  
- This process, called oxidative phosphorylation, produces most of the ATP generated from glucose.



Summary:  
Glucose → broken down into acetyl-CoA → fed into Krebs cycle → generates electron carriers (NADH, FADH₂) → electrons power proton pumping in ETC → protons flow back through ATP synthase → ATP is made.  

This efficient system